# Naver 영화별 Review Crawling

In [16]:
from urllib.request import urlopen
from bs4 import BeautifulSoup  

import pandas as pd

from tqdm import tqdm_notebook
import re

## Naver 영화 - 2020/03/01 평점순 영화별 제목, Review, 장르 Crawling

https://movie.naver.com/movie/sdb/rank/rmovie.nhn

In [17]:
domain = 'https://movie.naver.com'
story = []
title = []
genre = []

for i in tqdm_notebook( range ( 1, 31 ) ):
    url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20200301&page=" + str( i )
    html = urlopen( url )
    soup = BeautifulSoup( html, "lxml" )
    titles = soup.find_all( 'div', class_ = 'tit5' )
    hype = []
    href = []
  
    try:
        for each in titles:  
            hype = each.find_all( 'a' )
            for link in hype:
                href.append( link[ 'href' ] )
                
        for j in tqdm_notebook( range( len( href ) ) ):
            domain = 'https://movie.naver.com'
            domain = domain + href[ j ]
            html = urlopen( domain )
            soup = BeautifulSoup( html,  "lxml" )
            story.append( soup.find( 'p', class_ = "con_tx" ).get_text() )
            
            title_tag = soup.find( 'h3', class_ = 'h_movie' )
            title.append( title_tag.find( 'a' ).get_text() )
            
            genre_tag = soup.find( 'p' )
            genre.append( genre_tag.find( 'a' ).get_text() )
    except:
        pass
                  

# 줄거리 정규화 처리
for i in tqdm_notebook( range( len( story ) ) ):
    story[i] = re.sub( '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》“”’]', '', story[ i ] )
    story[i] = re.sub( '\r\xa0', '', story[ i ] )

d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


## Pandas DataFrame 생성

In [18]:
movie = pd.DataFrame( data = { 'title': title, 'story': story, 'genre': genre } )

In [19]:
movie.shape

(1107, 3)

In [20]:
movie.head()

,title,story,genre
0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마
1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마
2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험
3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리
4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션


In [21]:
movie.tail()

,title,story,genre
1102,개들의 섬,인류를 위협하는 개 독감이 퍼지자 세상의 모든 개들은 쓰레기 섬으로 추방되고 자신이...,애니메이션
1103,위대한 개츠비,1922년 뉴욕 외곽에서 살고 있는 닉은 호화로운 별장에 살고 있는 이웃 개츠비에게...,드라마
1104,샬롯의 거미줄,어느 봄 비바람치던 밤에 세상에 태어난 새끼 돼지 윌버 너무 조그맣다는 이유로 윌버...,코미디
1105,사쿠란,안노 모요코의 동명의 인기 만화를 영화화하는 작품벚꽃이 한창인 봄 8세 나이로 요시...,드라마
1106,"좋아해, 너를",수제 구두가게에서 일하는 레온최민기은 남들이 모르는 자신만의 아픈 상처로 인해 다른...,드라마


### 전체 장르 리스트

In [22]:
genres_list = movie[ 'genre' ].unique().tolist()
genres_list

['드라마',
 '모험',
 '다큐멘터리',
 '액션',
 'SF',
 '애니메이션',
 '범죄',
 '멜로/로맨스',
 '전쟁',
 '가족',
 '판타지',
 '코미디',
 '공연실황',
 '한국',
 '서부',
 '뮤지컬',
 '공포',
 '미스터리',
 '스릴러',
 '느와르',
 '서스펜스']

### 전체 장르에 대한 코드 부여

In [23]:
genre_code_dict = dict( ( c, i ) for i, c in enumerate( genres_list ) )
genre_code_dict

{'드라마': 0,
 '모험': 1,
 '다큐멘터리': 2,
 '액션': 3,
 'SF': 4,
 '애니메이션': 5,
 '범죄': 6,
 '멜로/로맨스': 7,
 '전쟁': 8,
 '가족': 9,
 '판타지': 10,
 '코미디': 11,
 '공연실황': 12,
 '한국': 13,
 '서부': 14,
 '뮤지컬': 15,
 '공포': 16,
 '미스터리': 17,
 '스릴러': 18,
 '느와르': 19,
 '서스펜스': 20}

### 전체 장르수 계산

In [24]:
max_genre_code_length = len( genre_code_dict )
max_genre_code_length

21

### 각 영화별 장르 코드 변수 추가

In [25]:
movie[ 'genre_code' ] = movie[ 'genre' ].map( lambda x : genre_code_dict[ x ] )

In [26]:
movie.head()

,title,story,genre,genre_code
0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마,0
1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마,0
2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험,1
3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리,2
4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션,3


In [28]:
movie.tail()

,title,story,genre,genre_code
1102,개들의 섬,인류를 위협하는 개 독감이 퍼지자 세상의 모든 개들은 쓰레기 섬으로 추방되고 자신이...,애니메이션,5
1103,위대한 개츠비,1922년 뉴욕 외곽에서 살고 있는 닉은 호화로운 별장에 살고 있는 이웃 개츠비에게...,드라마,0
1104,샬롯의 거미줄,어느 봄 비바람치던 밤에 세상에 태어난 새끼 돼지 윌버 너무 조그맣다는 이유로 윌버...,코미디,11
1105,사쿠란,안노 모요코의 동명의 인기 만화를 영화화하는 작품벚꽃이 한창인 봄 8세 나이로 요시...,드라마,0
1106,"좋아해, 너를",수제 구두가게에서 일하는 레온최민기은 남들이 모르는 자신만의 아픈 상처로 인해 다른...,드라마,0


## csv로 저장하기 

In [27]:
movie.to_csv( 'movie_reviews.csv' )